In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [7100, 12795, 20079, 664, 917, 5718, 11463, 15128, 6010, 15986, 18849, 3535, 3893, 5172, 12373, 17228, 15824, 519, 151, 14969, 16098, 7909, 2158, 10237, 17594, 3232, 5050, 4640, 4586, 10961, 14436, 520, 19059, 10229, 18492, 19629, 13934, 4578, 17126, 8089, 15060, 15079, 10122, 16237, 18467, 18560, 15341, 1151, 3230, 10442, 9740, 18182, 5739, 7441, 4537, 12522, 7902, 11547, 13200, 10608, 2741, 9359, 12005, 9247, 17386, 17924, 3540, 14513, 3396, 13340, 12128, 15999, 992, 4170, 6240, 4254, 12094, 4566, 2027, 18769, 4553, 11409, 15048, 14814, 17392, 2298, 8151, 10435, 15876, 3288, 7481, 12380, 1811, 17446, 17934, 15574, 18466, 19283, 8862, 7053, 1944, 7953, 8829, 19337, 7643, 2752, 15790, 1737, 18805, 9286, 9180, 10151, 17998, 18862, 13893, 15417, 10834, 5352, 13320, 30, 6710, 19186, 10488, 9688, 4172, 7908, 8657, 8232, 6408, 2620, 4428, 4534, 15669, 9411, 1002, 19706, 12515, 14422, 13132, 15203, 11823, 19896, 8890, 3413, 4937, 6632,

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'no', train_mode = '')

Epoch [1/500], Loss: 1.3900, Accuracy: 0.3471, Edge Accuracy: []
Class 0: AUC-ROC- 0.6102, F1 Score- 0.4545; Class 1: AUC-ROC- 0.3717, F1 Score- 0.2105; Class 2: AUC-ROC- 0.4245, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4577, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4660; Macro-Average F1 Score: 0.1663
Validation Loss: 1.3863, Validation Accuracy: 0.3750, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.8840, F1 Score- 0.4000; Class 1: AUC-ROC- 0.4356, F1 Score- 0.4000; Class 2: AUC-ROC- 0.4684, F1 Score- 0.0000; Class 3: AUC-ROC- 0.5001, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5720; Macro-Average F1 Score: 0.2000
Test Loss: 1.3873, Test Accuracy: 0.3800, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.8965, F1 Score- 0.4000; Class 1: AUC-ROC- 0.3791, F1 Score- 0.4000; Class 2: AUC-ROC- 0.4328, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4893, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5494; Macro-Average F1 Score: 0.2000

Epoch [2/500], Loss: 1.3833, Accuracy: 0.4059, Edge Accuracy: []
Class 0

In [8]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

TypeError: test_smote() missing 1 required positional argument: 'args'